In [1]:
from pandas import read_csv
# https://archive.ics.uci.edu/ml/datasets/Mushroom
srooms_df = read_csv('../data/agaricus-lepiota.data.csv')
srooms_df.head()

,edibility,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


## Feature extraction

If we wanted to use all the features in the training set then we would need to map each out.  The ```LabelEncoder``` converts T/F data to 1 and 0.  The ```LabelBinarizer``` converters categorical data to an **one hot encoding**. 

> Note: Bruises has missing data. 

If we wanted to use all the features in the training set then we would need to map each out:

```
column_names = srooms_df.axes[1]
def get_mapping(name):
    if(name == 'edibility' or name == 'gill-attachment'):
        return (name, sklearn.preprocessing.LabelEncoder())
    else:
        return (name, sklearn.preprocessing.LabelBinarizer())
    
mappings = list(map(lambda name: get_mapping(name), column_names)
```

We will use a subset of features to make it interesting.  Are there simple rules or a handful of features that can be used to test edibility?  Lets try a few...

In [12]:
from sklearn_pandas import DataFrameMapper
import sklearn
import numpy as np

mappings = ([
    ('edibility', sklearn.preprocessing.LabelEncoder()),
    ('odor', sklearn.preprocessing.LabelBinarizer()),
    ('habitat', sklearn.preprocessing.LabelBinarizer()),
    ('spore-print-color', sklearn.preprocessing.LabelBinarizer())
])

Now lets look at the transformed data.

In [13]:
mapper = DataFrameMapper(mappings)
srooms_np = mapper.fit_transform(srooms_df.copy())
print(srooms_np.shape)

(8124, 26)

In [14]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(srooms_np, test_size = 0.2, random_state=7)
train_labels = train[:,0:1]
train_data = train[:,1:]
test_labels = test[:,0:1]
test_data = test[:,1:]
print('training data dims: {}, label dims: {}'.format(train_data.shape,train_labels.shape))

training data dims: (6499, 25), label dims: (6499, 1)


In [15]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=25))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels, epochs=10, batch_size=32)

Epoch 1/10
6499/6499 [==============================] - 0s - loss: 0.4945 - acc: 0.7850     
Epoch 2/10
6499/6499 [==============================] - 0s - loss: 0.2195 - acc: 0.9448     
Epoch 3/10
6499/6499 [==============================] - 0s - loss: 0.1244 - acc: 0.9668     
Epoch 4/10
6499/6499 [==============================] - 0s - loss: 0.0936 - acc: 0.9720     
Epoch 5/10
6499/6499 [==============================] - 0s - loss: 0.0726 - acc: 0.9798     
Epoch 6/10
6499/6499 [==============================] - 0s - loss: 0.0655 - acc: 0.9789     
Epoch 7/10
6499/6499 [==============================] - 0s - loss: 0.0517 - acc: 0.9838     
Epoch 8/10
6499/6499 [==============================] - 0s - loss: 0.0435 - acc: 0.9863     
Epoch 9/10
6499/6499 [==============================] - 0s - loss: 0.0392 - acc: 0.9891     
Epoch 10/10
6499/6499 [==============================] - 0s - loss: 0.0341 - acc: 0.9914     


In [16]:
score = model.evaluate(test_data, test_labels, batch_size=1625)
print(score)

1625/1625 [==============================] - 0s
[0.011165722273290157, 0.99507689476013184]
